# Getting started

## Load functionality

In [1]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
# 
from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL, 
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL, 
    DEFAULT_USER_COL, 
)
#
from ebrec.utils._behaviors import (
    create_binary_labels_column, 
    sampling_strategy_wu2019,
    add_known_user_column,
    add_prediction_scores,
    truncate_history, 
)
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
#
from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel

/Users/nicolacecere/opt/anaconda3/envs/recsys2024/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-12 18:05:11.681529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load dataset

In [2]:
def ebnerd_from_path(path:Path, history_size:int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function 
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes, df2=df_history.collect(), on=DEFAULT_USER_COL, how="left"
        )
    )
    return df_behaviors

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [42]:
path = Path("../../../data/ebnerd_demo")
COLUMNS = [DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL, DEFAULT_INVIEW_ARTICLES_COL, DEFAULT_CLICKED_ARTICLES_COL]
HISTORY_SIZE = 30 # to tune
#N_SAMPLES = 100
df_train = (
    ebnerd_from_path(path.joinpath("train"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(sampling_strategy_wu2019,npratio=4,shuffle=True,with_replacement=True, seed=123)
    .pipe(create_binary_labels_column)
    #.sample(n=N_SAMPLES)
)
# =>
df_validation = (
    ebnerd_from_path(path.joinpath("validation"), history_size=HISTORY_SIZE)
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    #.sample(n=N_SAMPLES)
)
# =>
'''
df_test = (
    ebnerd_from_path(path.joinpath("test"), history_size=HISTORY_SIZE)
    .with_columns(pl.Series(DEFAULT_CLICKED_ARTICLES_COL, [[]]))
    .select(COLUMNS)
    .pipe(create_binary_labels_column)
    .sample(n=N_SAMPLES)
)
'''

/Users/nicolacecere/opt/anaconda3/envs/recsys2024/lib/python3.11/site-packages/ebrec/utils/_behaviors.py:619: UserWarning: truncate_history: The history IDs expeced in ascending order
  warnings.warn(f"{function_name}: The history IDs expeced in ascending order")


'\ndf_test = (\n    ebnerd_from_path(path.joinpath("test"), history_size=HISTORY_SIZE)\n    .with_columns(pl.Series(DEFAULT_CLICKED_ARTICLES_COL, [[]]))\n    .select(COLUMNS)\n    .pipe(create_binary_labels_column)\n    .sample(n=N_SAMPLES)\n)\n'

In [43]:
df_train

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,labels
u32,list[i32],list[i64],list[i64],list[i8]
22779,"[9768820, 9767534, … 9770541]","[9759544, 9775371, … 9774461]",[9759966],"[0, 0, … 0]"
150224,"[9767447, 9767739, … 9735909]","[9777397, 9778661, … 9778657]",[9778661],"[0, 1, … 0]"
160892,"[9759345, 9766042, … 9770178]","[9778155, 9778351, … 9778448]",[9777856],"[0, 0, … 0]"
1001055,"[9767220, 9766886, … 9769981]","[9776808, 9776855, … 9776808]",[9776566],"[0, 0, … 0]"
1001055,"[9767220, 9766886, … 9769981]","[9776553, 9776570, … 9776449]",[9776553],"[1, 0, … 0]"
1001055,"[9767220, 9766886, … 9769981]","[9776551, 9776508, … 9776544]",[9776508],"[0, 1, … 0]"
1001055,"[9767220, 9766886, … 9769981]","[9775985, 9776691, … 9488213]",[9776691],"[0, 1, … 0]"
1469458,"[9739411, 6998758, … 9764765]","[9776929, 9778158, … 7594265]",[9778158],"[0, 1, … 0]"
1469458,"[9739411, 6998758, … 9764765]","[9779227, 9695098, … 9779071]",[9779071],"[0, 0, … 1]"


### Look at the difference between Training/Validation and Testset
Note, the testset doesn't include labels, and we have remove some of the other columns.

In [15]:
df_train.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,labels
u32,list[i32],list[i64],list[i64],list[i8]
900378,"[9769380, 9769356, … 9770726]","[9770792, 9769762, … 9770886]",[9768308],"[0, 0, … 0]"
158641,"[9766805, 9766973, … 9769888]","[9766772, 9776041, … 9766772]",[9775915],"[0, 0, … 0]"


## Load articles

In [16]:
df_articles = pl.read_parquet(path.joinpath("articles.parquet"))
df_articles.head(2)

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3037230,"""Ishockey-spill…","""ISHOCKEY: Isho…",2023-06-29 06:20:57,false,"""Ambitionerne o…",2003-08-28 08:55:00,null,"""article_defaul…","""https://ekstra…",[],[],"[""Kriminalitet"", ""Kendt"", … ""Mindre ulykke""]",142,"[327, 334]","""sport""",null,null,null,0.9752,"""Negative"""
3044020,"""Prins Harry tv…","""Hoffet tvang P…",2023-06-29 06:21:16,false,"""Den britiske t…",2005-06-29 08:47:00,"[3097307, 3097197, 3104927]","""article_defaul…","""https://ekstra…","[""Harry"", ""James Hewitt""]","[""PER"", ""PER""]","[""Kriminalitet"", ""Kendt"", … ""Personfarlig kriminalitet""]",414,[432],"""underholdning""",null,null,null,0.7084,"""Negative"""


## Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


article\_mapping maps the article id to the title embneddings whose lenght is pre-defined by the parameter. (truncation)

In [17]:
TRANSFORMER_MODEL_NAME = "bert-base-multilingual-cased"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]
MAX_TITLE_LENGTH = 30

# LOAD HUGGINGFACE:
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

# We'll init the word embeddings using the 
word2vec_embedding = get_transformers_word_embeddings(transformer_model)
# 
df_articles, cat_cal = concat_str_columns(df_articles, columns=TEXT_COLUMNS_TO_USE)
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)
# =>
article_mapping = create_article_id_to_value_mapping(df=df_articles, value_col=token_col_title)

In [34]:
df_articles['subtitle-title_encode_bert-base-multilingual-cased'][0].shape

(30,)

In [29]:
article_mapping[9768308]

""
i64
106306
10172
177
156
118
15694
12536
86008
119


# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

In [35]:
train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=64,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=32,
)
# test_dataloader = NRMSDataLoader(
#     behaviors=df_test,
#     article_dict=article_mapping,
#     unknown_representation="zeros",
#     history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
#     eval_mode=True,
#     batch_size=32,
# )

## Train the model


In [38]:
import os
os.listdir()

['nrms_docvec_dummy.py',
 'lstur_dummy.py',
 'nrms_ebnerd.ipynb',
 'npa_dummy.py',
 'nrms_dummy.py',
 'naml_dummy.py',
 'data',
 'dataset_ebnerd.ipynb']

In [39]:
MODEL_NAME = "NRMS"
LOG_DIR = f"data/result_experiments/log/runs/{MODEL_NAME}"
MODEL_WEIGHTS = f"data/result_experiments/state_dict/{MODEL_NAME}/weights"

# CALLBACKS
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS, save_best_only=True, save_weights_only=True, verbose=1
)

hparams_nrms.history_size = HISTORY_SIZE
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=1,
    callbacks=[tensorboard_callback, early_stopping, modelcheckpoint],
)
#model.model.load_weights(filepath=MODEL_WEIGHTS)

2/2 [==============================] - ETA: 0s - loss: 1.6096
Epoch 1: val_loss improved from inf to 0.00000, saving model to data/result_experiments/state_dict/NRMS/weights
2/2 [==============================] - 208s 202s/step - loss: 1.6096 - val_loss: 0.0000e+00


In [41]:
model.model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, None, 30)]           0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 30, 30)]             0         []                            
                                                                                                  
 time_distributed_3 (TimeDi  (None, None, 400)            9281409   ['input_7[0][0]']             
 stributed)                                               6                                       
                                                                                                  
 user_encoder (Functional)   (None, 400)                  9337449   ['input_6[0][0]']       

# Example how to compute some metrics:

In [78]:
pred_validation = model.scorer.predict(val_dataloader)

4/4 [==============================] - 29s 5s/step


In [85]:
df_validation = add_prediction_scores(df_validation, pred_validation.tolist()).pipe(
    add_known_user_column, known_users=df_train[DEFAULT_USER_COL]
)
df_validation.head(2)

user_id,article_id_fixed,article_ids_inview,article_ids_clicked,labels,scores,is_known_user
u32,list[i32],list[i32],list[i32],list[i8],list[f64],bool
900378,"[9778842, 9778769, … 9780195]","[9482970, 9781785, … 9779066]",[9781785],"[0, 1, … 0]","[0.499117, 0.500279, … 0.49903]",false
918965,"[9764070, 9761801, … 9777026]","[9783732, 9533957, … 7184889]",[9783904],"[0, 0, … 0]","[0.500605, 0.499671, … 0.499687]",false


In [93]:
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
metrics = MetricEvaluator(
    labels=df_validation["labels"].to_list(),
    predictions=df_validation["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(),  NdcgScore(k=5),  NdcgScore(k=10)],
)
metrics.evaluate()

<MetricEvaluator class>: 
 {
    "auc": 0.49090675999255273,
    "mrr": 0.3155876682685506,
    "ndcg@5": 0.3357636766751604,
    "ndcg@10": 0.4161786339827512
}

## Make submission file

In [ ]:
pred_test = model.scorer.predict(test_dataloader)

array([[0.5000621 ],
       [0.49848208],
       [0.49827188],
       ...,
       [0.5002804 ],
       [0.4998887 ],
       [0.49926934]], dtype=float32)